<a href="https://colab.research.google.com/github/erendagasan/Eren-Dagasan-Personal/blob/main/UI_DENEME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
#@title Programı ilk açtığınızda mutlaka bir kere çalıştırın.
!pip install -q bta-lib
!pip install -q ta
!pip install -q yfinance

import yfinance as yf
import btalib
import numpy as np
import pandas as pd
from ta.trend import PSARIndicator
from ta.momentum import WilliamsRIndicator
from ta.trend import AroonIndicator
from ta.volume import VolumePriceTrendIndicator
from ta.trend import CCIIndicator
from ta.momentum import ROCIndicator
from ta.trend import ADXIndicator
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
import random
import numpy as np
import warnings
from google.colab import drive

warnings.filterwarnings("ignore")
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_rows', 500)

def create_indicators(data):
  # 15 DKLIK İCİN
  # EMA 33, 165, 660

  # ICHIMOKU
  # MOMENTUM INDIKATORU EKLE
  # ELLIOT EKLE
  # FIBONACCI

  # 50 - 200Ü KIRIYORSA GOLDEN CROSS - TAM TERSİ SAĞLAM SAT (DATA COK BÜYÜYOR)

  # Below 30 is oversold, upper 70 is overbought
  data["RSI"] = btalib.rsi(data["Close"], period=14).df

  #If little period conflicts with the bigger one it means buy else sell.
  data["SMA5"] = btalib.sma(data['Close'], period=5).df
  data["SMA9"] = btalib.sma(data['Close'], period=9).df
  data["SMA10"] = btalib.sma(data['Close'], period=10).df
  data["SMA14"] = btalib.sma(data['Close'], period=14).df
  data["SMA20"] = btalib.sma(data['Close'], period=20).df
  data["SMA21"] = btalib.sma(data['Close'], period=21).df
  data["SMA50"] = btalib.sma(data['Close'], period=50).df
  data["SMA200"] = btalib.sma(data['Close'], period=200).df

  data["EMA5"] = btalib.ema(data['Close'], period=5).df
  data["EMA14"] = btalib.ema(data['Close'], period=14).df
  data["EMA21"] = btalib.ema(data['Close'], period=21).df
  data["EMA50"] = btalib.ema(data['Close'], period=50).df

  #A reading below 20 generally represents an oversold market and a reading above 80 an overbought market. Look only STOCH-D.
  data["STOCH-K"] = btalib.stoch(data['High'], data['Low'], data['Close']).df["k"]
  data["STOCH-D"] = btalib.stoch(data['High'], data['Low'], data['Close']).df["d"]

  #When the MACD line crosses above the signal line, it generates a buy signal. This indicates a potential bullish trend reversal.
  #Additionally, monitor the MACD histogram. When the histogram bars turn positive (change from negative to positive), it confirms the buy signal and suggests increasing buying pressure.
  data["MACD"] = btalib.macd(data['Close']).df["macd"]
  data["SIGNAL"] = btalib.macd(data['Close']).df["signal"]
  data["HISTOGRAM"] = btalib.macd(data['Close']).df["histogram"]

  #Buy Signal: When the price of the stock touches or crosses below the lower Bollinger Band, it can be considered a buy signal.
  #This suggests that the stock may be oversold, and a potential price rebound is expected.
  #Sell Signal: When the price of the stock touches or crosses above the upper Bollinger Band, it can be considered a sell signal.
  #This indicates that the stock may be overbought, and a potential price correction is expected.
  data["BB-UPPER"] = btalib.bbands(data['Close']).df['top']
  data["BB-MID"] = btalib.bbands(data['Close']).df['mid']
  data["BB-LOWER"] = btalib.bbands(data['Close']).df['bot']

  #When the current price of the stock falls below a certain number of standard deviations (GET THE MIN AND MAX VALUE OF THE COLUMN AND MAKE IT THE THRESHOLD)
  #from the mean, it generates a buy signal. This suggests that the price has deviated significantly from the average and may present a buying opportunity.
  data["STDEV"] = data["Close"].rolling(window=10).std()

  #Generate a buy signal when the price crosses above the PSAR.
  #This indicates a potential reversal in the downward trend and suggests a buying opportunity.
  data["PSAR"] = PSARIndicator(data["High"], data["Low"], data["Close"]).psar()

  #Williams
  #Use the Williams Percent Range (%R) formula to calculate the %R values based on historical price data.
  #The %R values range from -100 to 0, where values close to -100 indicate oversold conditions and values close to 0 indicate overbought conditions.
  #Generate a buy signal when the %R value crosses above a certain threshold (e.g., -80) from below.
  #This indicates a potential reversal from oversold conditions and suggests a buying opportunity.
  data["WILLIAMS"] = WilliamsRIndicator(data["High"], data["Low"], data["Close"]).williams_r()

  #The difference between these two lines indicates whether there is overbought (a positive number) or oversold (a negative number).
  data["AROON"] = AroonIndicator(close=data["Close"], window=25).aroon_indicator()

  #obv
  data["OBV"] = VolumePriceTrendIndicator(close=data['Close'], volume=data['Volume']).volume_price_trend()

  #CCI
  data['CCI'] = CCIIndicator(close=data['Close'], low=data["Low"], high=data["High"], window=14).cci()

  #ROC
  data['ROC'] = ROCIndicator(close=data['Close'], window=5).roc()

  #BULLBEARPOWER
  data['BULL'] = data['High'] - (data['High'].rolling(13).max() + data['Low'].rolling(13).min()) / 2
  data['BEAR'] = data['Low'] - (data['High'].rolling(13).max() + data['Low'].rolling(13).min()) / 2

  #ADX
  adx_indicator = ADXIndicator(high=data['High'], low=data['Low'], close=data['Close'], window=14)
  data['ADX'] = adx_indicator.adx()
  data['+DI'] = adx_indicator.adx_pos()
  data['-DI'] = adx_indicator.adx_neg()

  data = data.dropna()
  data = data.reset_index()
  return data

def buy_or_sell(stock, data, balance, upper_score, lower_score):
  stock_balance = 0
  transactions = []

  for row in range(data.shape[0]):
    if row > 0 and data["SCORE"].iloc[row] >= upper_score and data["SCORE"].iloc[row-1] < upper_score and balance > data["Close"].iloc[row]:
      stock_balance = balance / data["Close"].iloc[row]
      balance-= stock_balance*data["Close"].iloc[row]
      transactions.append([stock, "BUY", data["Date"].iloc[row], data["Close"].iloc[row], stock_balance, balance, data["SCORE"].iloc[row]])

    elif row > 0 and data["SCORE"].iloc[row] <= lower_score and data["SCORE"].iloc[row-1] > lower_score and stock_balance > 1:
      balance+=stock_balance*data["Close"].iloc[row]
      stock_balance-=stock_balance
      transactions.append([stock,"SELL", data["Date"].iloc[row], data["Close"].iloc[row], stock_balance, balance, data["SCORE"].iloc[row]])

  transactions_df = pd.DataFrame(transactions, columns=["STOCK","B/S", "DATE", "PRICE", "STOCK BALANCE", "TRY BALANCE", "SCORE"])
  if transactions_df.empty != True:
    transactions_df["LAST PRICE"] = data["Close"].iloc[-1]
  return transactions_df

def test_the_market(market, buy_threshold, sell_threshold, start, end, balance, today, x):
  test_df = pd.DataFrame()

  for stock in market:
    try:
      data = yf.download(stock, start, end, progress=False)
      data = create_indicators(data)
      data = score(data, x)
      simulation_df = buy_or_sell(stock, data, balance=balance, upper_score=buy_threshold, lower_score=sell_threshold)
      test_df = pd.concat([test_df, simulation_df])

    except:
      print("Data error with stock named: " + stock)

  test_df = test_df.reset_index().drop(["index"], axis=1)
  return_df = pd.DataFrame()

  for stock in market:
    stock_df = test_df[test_df["STOCK"] == stock]

    if stock_df.empty != True:
      if stock_df['B/S'].iloc[-1] == "BUY":
        add = pd.DataFrame({"STOCK": [stock_df["STOCK"].iloc[-1]], "B/S": ["CLOSE"], "DATE": [today], "PRICE": [stock_df["LAST PRICE"].iloc[-1]],
                            'STOCK BALANCE': [0], 'TRY BALANCE': [(stock_df["STOCK BALANCE"].iloc[-1]) * stock_df["LAST PRICE"].iloc[-1]], 'LAST PRICE' :[stock_df["LAST PRICE"].iloc[-1]]})

        stock_df = stock_df.append(add)
        return_df = pd.concat([return_df, stock_df], ignore_index=True)
      else:
        return_df = pd.concat([return_df, stock_df], ignore_index=True)

  return return_df

def score(data, x):
  scaler = MinMaxScaler(feature_range=(0, 100))
  scores = []

  x = ast.literal_eval(x)

  for row in range(data.shape[0]):
    score = 0

    if data["RSI"][row] < 30:
      score+=x[0]
    elif data["RSI"][row] > 70:
      score-=x[1]

    if data["RSI"][row] >= 75:
      score+=x[58]
    elif row > 0 and data["RSI"][row-1] > 73 and data["RSI"][row] < 73:
      score-=x[59]

    if row > 0 and data["RSI"].iloc[row-1] > 63 and data["RSI"].iloc[row] < 63:
      score-=x[2]

    if row > 0 and data["RSI"].iloc[row-1] > 49 and data["RSI"].iloc[row] < 49:
      score-=x[3]
    elif row > 0 and data["RSI"].iloc[row-1] < 50 and data["RSI"].iloc[row] > 50:
      score+=x[4]

    if data["SMA5"].iloc[row] > data["SMA10"].iloc[row]:
      score+=x[5]
    else:
      score-=x[6]

    if row > 0 and data['SMA5'].iloc[row-1] < data['SMA10'].iloc[row-1] and data['SMA5'].iloc[row] > data['SMA10'].iloc[row]:
      score+=x[7]
    else:
      score-=x[8]

    if data["SMA9"][row] > data["SMA14"][row]:
      score+=x[9]
    else:
      score-=x[10]

    if row > 0 and data['SMA9'].iloc[row-1] < data['SMA14'].iloc[row-1] and data['SMA9'].iloc[row] > data['SMA14'].iloc[row]:
      score+=x[11]
    else:
      score-=x[12]

    if data["SMA14"][row] > data["SMA21"][row]:
      score+=x[13]
    else:
      score-=x[14]

    if row > 0 and data['SMA14'].iloc[row-1] < data['SMA21'].iloc[row-1] and data['SMA14'].iloc[row] > data['SMA21'].iloc[row]:
      score+=x[15]
    else:
      score-=x[16]

    if row > 1 and data['SMA20'].iloc[row-1] < data["Close"].iloc[row-1] and data['SMA20'].iloc[row] < data["Close"].iloc[row]:
      score+=x[17]
    else:
      score-=x[18]

    if data["SMA21"][row] > data["SMA50"][row]:
      score+=x[19]
    else:
      score-=x[20]

    if row > 0 and data['SMA21'].iloc[row-1] < data['SMA50'].iloc[row-1] and data['SMA21'].iloc[row] > data['SMA50'].iloc[row]:
      score+=x[21]
    else:
      score-=x[22]

    if row > 0 and data['SMA50'].iloc[row-1] < data['SMA200'].iloc[row-1] and data['SMA50'].iloc[row] > data['SMA200'].iloc[row]:
      score+=x[23]

    if data['Close'][row] > data["EMA14"][row]:
      score+=x[24]
    else:
      score-=x[25]

    if data["EMA5"][row] > data["EMA14"][row]:
      score+=x[26]
    else:
      score-=x[27]

    if row > 0 and data['EMA5'].iloc[row-1] < data['EMA14'].iloc[row-1] and data['EMA5'].iloc[row] > data['EMA14'].iloc[row]:
      score+=x[28]
    else:
      score-=x[29]

    if data["EMA14"][row] > data["EMA21"][row]:
      score+=x[30]
    else:
      score-=x[31]

    if row > 0 and data['EMA14'].iloc[row-1] < data['EMA21'].iloc[row-1] and data['EMA14'].iloc[row] > data['EMA21'].iloc[row]:
      score+=x[32]
    else:
      score-=x[33]

    if data["EMA21"][row] > data["EMA50"][row]:
      score+=x[34]
    else:
      score-=x[35]

    if row > 0 and data['EMA21'].iloc[row-1] < data['EMA50'].iloc[row-1] and data['EMA21'].iloc[row] > data['EMA50'].iloc[row]:
      score+=x[36]
    else:
      score-=x[37]

    if data["STOCH-D"][row] <= 20:
      score+=x[38]
    elif data["STOCH-D"][row] >= 80:
      score-=x[39]

    if row > 0 and data["STOCH-D"][row-1] > 65 and data["STOCH-D"][row] < 65:
      score-=x[40]
    elif row > 0 and data["STOCH-D"][row-1] < 35 and data["STOCH-D"][row] > 35:
      score=+x[41]

    if data["MACD"][row] > data["SIGNAL"][row]:
      score+=x[42]
    if row > 0 and data["MACD"][row] > data["SIGNAL"][row] and data["HISTOGRAM"][row-1] < 0 and data["HISTOGRAM"][row] > 0:
      score+=x[43]
    if row > 0 and data["MACD"][row] < data["SIGNAL"][row] and data["HISTOGRAM"][row-1] > 0 and data["HISTOGRAM"][row] < 0:
      score-=x[44]
    if data["MACD"][row] < data["SIGNAL"][row]:
      score-=x[45]

    if data['MACD'].tail(7).mean() > 0:
      score+=x[46]
    else:
      score-=x[47]

    if data['MACD'].tail(30).mean() > 0:
      score+=x[48]
    else:
      score-=x[49]

    if data["BB-LOWER"].iloc[row] > data["Close"].iloc[row]:
        score+=x[50]
    elif data["BB-UPPER"].iloc[row] < data["Close"].iloc[row]:
        score-=x[51]
    if data["BB-MID"].iloc[row] < data["Close"].iloc[row] and data["BB-LOWER"].iloc[row] - data["Close"].iloc[row] < data["BB-MID"].iloc[row] - data["Close"].iloc[row]:
        score+=x[52]
    elif data["BB-MID"].iloc[row] > data["Close"].iloc[row] and data["BB-UPPER"].iloc[row] - data["Close"].iloc[row] < data["BB-MID"].iloc[row] - data["Close"].iloc[row]:
        score-=x[53]

    if (data["STDEV"].max() + data["STDEV"].min()) / 2 > data["STDEV"][row]:
      score+=1
    else:
      score-=1

    if data["PSAR"][row] < data["Close"][row]:
      score+=1
    else:
      score-=1

    if data["WILLIAMS"][row] < -80:
      score+=1
    elif data["WILLIAMS"][row] > -20:
      score-=1

    if data["AROON"][row] > 0:
      score-=1
    else:
      score+=1

    if row > 0 and data["OBV"][row-1] < data["OBV"][row] and data["Close"][row] > data["Close"][row-1]:
      score+=1
    elif row > 0 and data["OBV"][row-1] > data["OBV"][row] and data["Close"][row] < data["Close"][row-1]:
      score-=1

    if row > 0 and (data['CCI'][row] > -100) & (data['CCI'][row-1] <= -100):
      score+=1
    elif row > 0 and (data['CCI'][row] < 100) & (data['CCI'][row-1] >= 100):
      score-=1

    if data["ROC"][row] > 0:
      score+=1
    elif data["ROC"][row] < 0:
      score-=1

    if data["BULL"][row] > 0:
      score+=1
    elif data["BEAR"][row] < 0:
      score-=1

    if (data['ADX'][row] > 25) and (data['+DI'][row] > data['-DI'][row]):
      score+=x[54]
    if data['ADX'][row] > data['+DI'][row] and data['ADX'][row] > data['-DI'][row]:
      score+=x[55]
    if data['ADX'][row] < data['+DI'][row] and data['ADX'][row] < data['-DI'][row]:
      score-=x[56]
    elif (data['ADX'][row] > 40) and (data['+DI'][row] < data['-DI'][row]):
      score-=x[57]

    scores.append(score)

  data["SCORE"] = scores
  data["SCORE"] = scaler.fit_transform(data["SCORE"].values.reshape(-1, 1))
  return data

result_df = pd.read_excel("https://docs.google.com/spreadsheets/d/1RSqOXkFTAO7g4H9LEY3d3IX6H6bJaYk1/edit?usp=sharing&ouid=107922828925304876821&rtpof=true&sd=true")

today = datetime.today().date()
tomorrow = today + timedelta(days=1)

start="2021-06-01"
end = tomorrow.strftime("%Y-%m-%d")

balance = 2000

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


ValueError: ignored

In [28]:
#@title AL veya SAT durumunda olan hisseleri görmek için çalıştırın.
def color_boolean(row):
    color = ''

    if row["BUY OR SELL"] == "SELL":
        color = 'red'
    elif row["BUY OR SELL"] == "BUY":
        color = 'green'

    return ['background-color: %s' % color] * len(row)

buy_stocks = []
sell_stocks = []

for stock in result_df["STOCK"].unique():

  stock_df = result_df[result_df["STOCK"] == stock].sort_values(by="PROFIT", ascending=False)
  buy_threshold = stock_df["BUY TH"].iloc[0]
  sell_threshold = stock_df["SELL TH"].iloc[0]
  x = stock_df["WEIGHTS"].iloc[0]

  try:
    data = yf.download(stock, start=start, end=end, progress=False)
    data = create_indicators(data)
    data = score(data, x)

    if data["SCORE"].iloc[-1] > buy_threshold and data["SCORE"].iloc[-2] < buy_threshold:
      buy_stocks.append([data["Date"].iloc[-1], stock, data["Close"].iloc[-1], data["SCORE"].iloc[-1], "BUY", "%" + str(round(((data["Close"].iloc[-1]-data["Close"].iloc[-2])/data["Close"].iloc[-2])*100, 2))])

    elif data["SCORE"].iloc[-1] < sell_threshold and data["SCORE"].iloc[-2] > sell_threshold:
      sell_stocks.append([data["Date"].iloc[-1], stock, data["Close"].iloc[-1], data["SCORE"].iloc[-1], "SELL", "%" + str(round(((data["Close"].iloc[-1]-data["Close"].iloc[-2])/data["Close"].iloc[-2])*100, 2))])

  except:
    print("Data error at the stock named: " + stock)

buy_df = pd.DataFrame(buy_stocks, columns=["DATE", "STOCK", "PRICE", "SCORE", "BUY OR SELL", "CHANGE"])
sell_df = pd.DataFrame(sell_stocks, columns=["DATE", "STOCK", "PRICE", "SCORE", "BUY OR SELL", "CHANGE"])
whole_df = pd.concat([buy_df, sell_df], ignore_index=True)


whole_df = whole_df.style.apply(color_boolean, axis=1).format({col: "{:.2f}" for col in whole_df.select_dtypes(include='float').columns})
whole_df

,DATE,STOCK,PRICE,SCORE,BUY OR SELL,CHANGE
0,2023-07-07 00:00:00,TUPRS.IS,90.80,65.22,BUY,%3.12
1,2023-07-07 00:00:00,YKBNK.IS,12.96,69.84,BUY,%-0.46
2,2023-07-07 00:00:00,PETKM.IS,15.93,54.90,SELL,%0.0
3,2023-07-07 00:00:00,SISE.IS,46.56,59.72,SELL,%-0.3


In [26]:
#@title Geçmiş AL-SAT verilerini görmek istediğiniz hisse adını girin
Hisse_adi = "YKBNK" #@param {type:"string"}
stock = Hisse_adi.upper() + ".IS"

if stock in result_df["STOCK"].unique():
  profit_or_loss = []
  transactions_df = pd.DataFrame()

  stock_df = result_df[result_df["STOCK"] == stock].sort_values(by="PROFIT", ascending=False)

  buy_threshold = stock_df["BUY TH"].iloc[0]
  sell_threshold = stock_df["SELL TH"].iloc[0]
  x = stock_df["WEIGHTS"].iloc[0]

  test_df = test_the_market([stock], buy_threshold, sell_threshold, start=start, end=end, balance=balance, today=today, x=x)
  transactions_df = pd.concat([transactions_df, test_df], ignore_index=True)

  stock_df = test_df[test_df["STOCK"] == stock]

  if stock_df.empty != True:
    profit_or_loss.append([stock_df["STOCK"].iloc[-1] ,stock_df['TRY BALANCE'].tail(1).item() - balance])

  profit_df = pd.DataFrame(profit_or_loss)

def color_boolean(row):
    color = ''

    if row["B/S"] == "SELL":
        color = 'red'
    elif row["B/S"] == "BUY":
        color = 'green'
    elif row["B/S"] == 'CLOSE':
        color = 'blue'

    return ['background-color: %s' % color] * len(row)

transactions_df[transactions_df["STOCK"] == stock].style.apply(color_boolean, axis=1).format({col: "{:.2f}" for col in transactions_df.select_dtypes(include='float').columns})

,STOCK,B/S,DATE,PRICE,STOCK BALANCE,TRY BALANCE,SCORE,LAST PRICE
0,YKBNK.IS,BUY,2022-03-21 00:00:00,4.00,500.00,0.00,77.78,12.96
1,YKBNK.IS,SELL,2022-03-23 00:00:00,4.02,0.00,2010.00,68.25,12.96
2,YKBNK.IS,BUY,2022-03-29 00:00:00,4.00,502.50,0.00,69.84,12.96
3,YKBNK.IS,SELL,2022-04-26 00:00:00,4.94,0.00,2482.35,53.97,12.96
4,YKBNK.IS,BUY,2022-05-30 00:00:00,4.55,545.57,0.00,66.67,12.96
5,YKBNK.IS,SELL,2022-06-02 00:00:00,4.85,0.00,2646.02,63.49,12.96
6,YKBNK.IS,BUY,2022-06-03 00:00:00,4.84,546.70,0.00,76.19,12.96
7,YKBNK.IS,SELL,2022-06-06 00:00:00,4.98,0.00,2722.56,65.08,12.96
8,YKBNK.IS,BUY,2022-06-10 00:00:00,4.75,573.17,0.00,68.25,12.96
9,YKBNK.IS,SELL,2022-06-21 00:00:00,4.98,0.00,2854.39,68.25,12.96
